In [1]:
%matplotlib inline

%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np

import xarray as xr

import pandas as pd

import zarr

import sys
sys.path.append('../')
import utils

np.seterr(invalid='ignore'); # disable a warning from matplotlib and cartopy

# Data

## GEOS Chem Runs

Using egrid, NEI, and our model emissions inventories for NO, NO2, SO2

In [3]:
egrid_path = '../../GCrundirs/type_NA-out_conc-yr_2016-gridmod_final-validation-egrid/OutputDir/'
NEI_path = '../../GCrundirs/type_NA-out_conc-yr_2016-gtropchem/merra2_05x0625_tropchem_na/OutputDir/'
MODEL_path = '../../GCrundirs/type_NA-out_conc-yr_2016-gridmod_final-validation/OutputDir/'
nonuc_path = '../../GCrundirs/type_NA-out_conc-yr_2016-gridmod_nonuc/OutputDir/'
speciesconc_output = 'GEOSChem.SpeciesConc.2016*.nc4'
aerosol_output = 'GEOSChem.AerosolMass.2016*.nc4'

ds_egrid= xr.Dataset()
ds_egrid = utils.import_GC_runs_general(ds_egrid, egrid_path, aerosol_output, speciesconc_output, 'egrid')

ds_MODEL= xr.Dataset()
ds_MODEL = utils.import_GC_runs_general(ds_MODEL, MODEL_path, aerosol_output, speciesconc_output, 'MODEL')

ds_NEI= xr.Dataset()
ds_NEI = utils.import_GC_runs_general(ds_NEI, NEI_path, aerosol_output, speciesconc_output, 'NEI')


In [10]:
datasets = [
    ds_egrid,
    ds_NEI,
    ds_MODEL
]

In [11]:
ds = xr.concat(datasets, pd.Index(['egrid', 'NEI', 'MODEL'], name='model_name'))

In [12]:
#molecular weights of species
mw_species = {
    'NH4': 18.,
    'NIT': 62.,
    'SO4': 96.,
    'BCPI': 12.,
    'OCPI': 12.,
    'BCPO': 12.,
    'OCPO': 12.,
    'DST1': 29.,
    'DST2': 29.,
    'SALA': 31.4,
    'HNO3': 63.01,
    'NH3' : 17.031
} #g/mol
#list of pm species
pm_species = list(mw_species.keys())
#list of names of species in ds
pm_tracers = ['SpeciesConc_' + spec for spec in pm_species]

#loop through each model and create a subset for pm
pm_data = (
    ds[pm_tracers]
    .rename({'SpeciesConc_' + spec: spec for spec in pm_species})
)
pm_data = pm_data.isel(lev = 0)

In [13]:
xr.Dataset.to_zarr(pm_data, 'pm_ds.zarr', mode = 'w') #save the dataset 


PM25     = 1.33 (NH4 + NIT  + SO4)
         + BCPI + BCPO
         + 2.1 (OCPO + 1.16 OCPI) + 1.16 SOA
         + DST1 + 0.38 DST2 + 1.86 SALA